In [163]:
# from selenium import webdriver as wd
import chromedriver_autoinstaller
import threading
import requests
import queue
import time
import random
import pandas as pd
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup
import seleniumwire.undetected_chromedriver.v2 as wd
# from seleniumwire import webdriver as wd


chromedriver_autoinstaller.install()  

productname = "Ryzen 9"

url = "https://www.amazon.in/gp/new-releases/?ref_=nav_em_cs_newreleases_0_1_1_3"

options = {
    'proxy': {
        "http": "", 
        "https": "",
    }
}

driver = wd.Chrome(seleniumwire_options = options)
time.sleep(8)
req = driver.get(url)
time.sleep(8)
search_bar = driver.find_element(By.ID, "twotabsearchtextbox")
search_bar.send_keys(productname)
search_ok_button = driver.find_element(By.ID, "nav-search-submit-button")
search_ok_button.click()


None


In [167]:
soup = BeautifulSoup(driver.page_source)

In [168]:
data_list = []

while(True):
    results = soup.findAll("div", {"data-component-type" : "s-search-result"})
    results

    for result in results:
        title = result.find("span", {"class" : "a-text-normal"})
        price = result.find("span", {"class" : "a-price-whole"})
        url = result.find("a", {"class": "a-link-normal"}, href = True)
       
        if price is not None:
            # print(title.text)
            # print(price.text)
            data_list.append([title.text, price.text, "https://www.amazon.in/"+url['href']])
            

    
    nextPageLink = driver.find_element(By.CLASS_NAME, "s-pagination-next")
    time.sleep(2)
    nextPageCSSLinks = nextPageLink.get_attribute("class").split()
    if not "s-pagination-disabled" in nextPageCSSLinks:
        nextPageLink.click()
        time.sleep(5)
    else:
        print("End of page")
        break
scraped_data_df = pd.DataFrame(data_list, columns=["title","price","url"])

End of page


In [169]:
scraped_data_df

,title,price,url
0,7000 Series Ryzen 9 7950X 3D Desktop Processor...,"66,499",https://www.amazon.in//AMD-Ryzen-Desktop-Proce...
1,5000 Series Ryzen 9 5950X Desktop Processor 16...,"49,990",https://www.amazon.in//AMD-Ryzen-5950X-Process...
2,5000 Series Ryzen 9 5900X Desktop Processor 12...,"32,159",https://www.amazon.in//AMD-Ryzen-5900X-Process...
3,7000 Series Ryzen 9 7900X Desktop Processor 12...,"39,999",https://www.amazon.in//AMD-7900X-Desktop-Proce...
4,7000 Series Ryzen 9 7950X Desktop Processor 16...,"59,899",https://www.amazon.in//AMD-7950X-Desktop-Proce...
...,...,...,...
75,5000 Series Ryzen 9 5950X Desktop Processor 16...,"82,149",https://www.amazon.in//AMD-Ryzen-Desktop-Proce...
76,7000 Series Ryzen 9 7900 Desktop Processor 12 ...,"43,699",https://www.amazon.in//AMD-Ryzen-7900-Processo...
77,7000 Series Ryzen 9 7900X 3D Desktop Processor...,"54,719",https://www.amazon.in//AMD-Ryzen-Desktop-Proce...
78,Ryzen Threadripper Pro Series Ryzen Threadripp...,"8,15,499",https://www.amazon.in//AMD-ThreadripperTM-5995...


In [170]:
# data cleaning
scraped_data_df['title'] = scraped_data_df['title'].str.lower()

#converting price column to int
scraped_data_df['price'] = scraped_data_df['price'].str.replace(',','').astype(float)

#removing irrelevant products
scraped_data_df[scraped_data_df['title'].str.contains('ryzen 9')]

,title,price,url
0,7000 series ryzen 9 7950x 3d desktop processor...,66499.0,https://www.amazon.in//AMD-Ryzen-Desktop-Proce...
1,5000 series ryzen 9 5950x desktop processor 16...,49990.0,https://www.amazon.in//AMD-Ryzen-5950X-Process...
2,5000 series ryzen 9 5900x desktop processor 12...,32159.0,https://www.amazon.in//AMD-Ryzen-5900X-Process...
3,7000 series ryzen 9 7900x desktop processor 12...,39999.0,https://www.amazon.in//AMD-7900X-Desktop-Proce...
4,7000 series ryzen 9 7950x desktop processor 16...,59899.0,https://www.amazon.in//AMD-7950X-Desktop-Proce...
...,...,...,...
73,5000 series ryzen 9 5950x ddr4 desktop process...,107990.0,https://www.amazon.in//AMD-Desktop-Processor-T...
74,3rdgen ryzen 9 3900x desktop processor 12 core...,56999.0,https://www.amazon.in//3rdGen-Ryzen-Desktop-Pr...
75,5000 series ryzen 9 5950x desktop processor 16...,82149.0,https://www.amazon.in//AMD-Ryzen-Desktop-Proce...
76,7000 series ryzen 9 7900 desktop processor 12 ...,43699.0,https://www.amazon.in//AMD-Ryzen-7900-Processo...
